## XGBoost Model For Rossmann Sales

In [21]:
import pandas as pd
import numpy as np
import pickle

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Loading and Viewing our Data

In [3]:
# Loading our data without feature engineering done
with open('data_without_time_vars.pickle', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.head()

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,IsHoliday,StoreType,Assortment,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,LogCompetitionDistance,StorePermutation,LogSales,CompetitionTenureMonths,LogCompetitionMonths,Promo2Months,LogPromo2Months
0,2015-07-31,1,5,5263.0,1,1,0,1,2015,7,31,31,1,3,1,9,2008,0,0,0,0,7.146772,7,8.568646,82,4.418841,0.0,0.0
1,2015-07-30,1,4,5020.0,1,1,0,1,2015,7,30,31,1,3,1,9,2008,0,0,0,0,7.146772,7,8.521384,82,4.418841,0.0,0.0
2,2015-07-29,1,3,4782.0,1,1,0,1,2015,7,29,31,1,3,1,9,2008,0,0,0,0,7.146772,7,8.472823,82,4.418841,0.0,0.0
3,2015-07-28,1,2,5011.0,1,1,0,1,2015,7,28,31,1,3,1,9,2008,0,0,0,0,7.146772,7,8.519590,82,4.418841,0.0,0.0
4,2015-07-27,1,1,6102.0,1,1,0,1,2015,7,27,31,1,3,1,9,2008,0,0,0,0,7.146772,7,8.716536,82,4.418841,0.0,0.0


In [5]:
def train_test_split(df, verbose=True):
    '''
    Function to split the dataframe into 2 sets of training and testing datasets for walk forward validation.
    '''

    trainset_1 = df[:'20141231']
    trainset_1.reset_index(drop=True, inplace=True)
    
    testset_1 = df['20150101':'20150331']
    testset_1.reset_index(drop=True, inplace=True)
    
    trainset_2 = df[:'20150331']
    trainset_2.reset_index(drop=True, inplace=True)
    
    testset_2 = df['20150401':]
    testset_2.reset_index(drop=True, inplace=True)

    if verbose:
        print('Initial dataset has been split into two sets of training and testing datasets each.')
        print('='*83)
        print('Trainset 1 is from 2013-01-02 to 2014-12-31 : N_obs = {}'.format(trainset_1.shape[0]))
        print('Testset 1 is from 2015-01-01 to 2015-03-31 : N_obs = {}'.format(testset_1.shape[0]))
        print('\n')
        print('Trainset 2 is from 2013-01-02 to 2015-03-31 : N_obs = {}'.format(trainset_2.shape[0]))
        print('Testset 2 is from 2015-04-01 to 2015-07-31 : N_obs = {}'.format(testset_2.shape[0]))

    return trainset_1, testset_1, trainset_2, testset_2

### Removing features that we are not needed.

In [6]:
df.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear', 'IsHoliday',
       'StoreType', 'Assortment', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'LogCompetitionDistance',
       'StorePermutation', 'LogSales', 'CompetitionTenureMonths',
       'LogCompetitionMonths', 'Promo2Months', 'LogPromo2Months'],
      dtype='object')

In [7]:
df.drop(['Sales', 'Promo2Months', 'CompetitionTenureMonths',
         'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
         'Promo2SinceWeek', 'Promo2SinceYear', ], axis=1, inplace=True)

In [8]:
df.dropna(inplace=True)

In [9]:
df = df.set_index('Date').sort_index(ascending=True)

In [10]:
df.columns

Index(['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'Year', 'Month', 'Day', 'WeekOfYear', 'IsHoliday', 'StoreType',
       'Assortment', 'Promo2', 'PromoInterval', 'LogCompetitionDistance',
       'StorePermutation', 'LogSales', 'LogCompetitionMonths',
       'LogPromo2Months'],
      dtype='object')

In [12]:
# Rearranging our features
df = df[['LogSales', 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday',
        'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear', 'StoreType',
        'Assortment', 'Promo2', 'PromoInterval', 'LogCompetitionDistance',
        'StorePermutation', 'LogCompetitionMonths', 'LogPromo2Months', 'IsHoliday']]

### We will only be looking at Category A stores.

In [13]:
df = df.loc[df.StoreType == 1]

In [14]:
df.head()

,LogSales,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,Promo2,PromoInterval,LogCompetitionDistance,StorePermutation,LogCompetitionMonths,LogPromo2Months,IsHoliday
Date,,,,,,,,,,,,,,,,,,,,
2013-01-01,8.752423,434,2,0,0,1,1,2013,1,1,1,1,1,1,1,9.474242,1,4.736198,0.000000,1
2013-01-01,9.118499,433,2,0,0,1,1,2013,1,1,1,1,3,0,0,10.309952,3,3.295837,0.000000,1
2013-01-01,9.223010,1009,2,0,0,1,1,2013,1,1,1,1,1,1,1,5.438079,1,4.634729,0.000000,1
2013-01-01,9.166232,432,2,0,0,1,1,2013,1,1,1,1,1,0,0,6.697034,1,0.000000,0.000000,1
2013-01-01,8.761315,427,2,0,0,1,1,2013,1,1,1,1,3,1,1,4.248495,3,4.510860,3.526361,1


### Splitting our Data into Training and Testing datasets.

In [15]:
trainset_1, testset_1, trainset_2, testset_2 = train_test_split(df)

Initial dataset has been split into two sets of training and testing datasets each.
Trainset 1 is from 2013-01-02 to 2014-12-31 : N_obs = 362920
Testset 1 is from 2015-01-01 to 2015-03-31 : N_obs = 46275


Trainset 2 is from 2013-01-02 to 2015-03-31 : N_obs = 409195
Testset 2 is from 2015-04-01 to 2015-07-31 : N_obs = 63105


In [16]:
y_train_1 = np.array(trainset_1.LogSales)
x_train_1 = np.array(trainset_1.drop('LogSales', axis=1))

y_train_2 = np.array(trainset_2.LogSales)
x_train_2 = np.array(trainset_2.drop('LogSales', axis=1))

y_test_1 = np.array(testset_1.LogSales)
x_test_1 = np.array(testset_1.drop('LogSales', axis=1))

y_test_2 = np.array(testset_2.LogSales)
x_test_2 = np.array(testset_2.drop('LogSales', axis=1))

In [17]:
# Creating XGBoost model #
gbm = xgb.XGBRegressor(n_estimators=10000,
                       max_dept=8,
                       objective="reg:linear",
                       learning_rate=0.05,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       n_jobs=-1,
                       random_state=8888)

In [18]:
# Tracking evaluation error for training and testing set 1 
eval_set_1 = [(x_train_1, y_train_1),
              (x_test_1, y_test_1)]

In [19]:
gbm_fit = gbm.fit(x_train_1,
                  y_train_1,
                  eval_set=eval_set_1,
                  eval_metric='rmse',
                  early_stopping_rounds=50,
                  verbose=True)

[16:56:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:7.84375	validation_1-rmse:7.83039
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:7.45258	validation_1-rmse:7.43889
[2]	validation_0-rmse:7.08114	validation_1-rmse:7.06748
[3]	validation_0-rmse:6.72822	validation_1-rmse:6.71417
[4]	validation_0-rmse:6.39302	validation_1-rmse:6.37858
[5]	validation_0-rmse:6.07463	validation_1-rmse:6.05984
[6]	validation_0-rmse:5.77222	validation_1-rmse:5.75679
[7]	validation_0-rmse:5.48501	validation_1-rmse:5.46927
[8]	validation_0-rmse:5.21221	validation_1-rmse:5.19589
[9]	validation_0-rmse:4.95311	validation_1-rmse:4.93645
[10]	validation_0-rmse:4.70706	validation_1-rmse:4.68986
[11]	validation_0-rmse:4.47337	validation_1-rmse:4.45576
[12]	validation_0-rmse:4.25145	validatio

[136]	validation_0-rmse:0.350061	validation_1-rmse:0.342963
[137]	validation_0-rmse:0.349419	validation_1-rmse:0.342385
[138]	validation_0-rmse:0.349275	validation_1-rmse:0.3423
[139]	validation_0-rmse:0.349012	validation_1-rmse:0.342009
[140]	validation_0-rmse:0.348846	validation_1-rmse:0.341859
[141]	validation_0-rmse:0.348656	validation_1-rmse:0.341691
[142]	validation_0-rmse:0.348149	validation_1-rmse:0.341201
[143]	validation_0-rmse:0.34801	validation_1-rmse:0.341207
[144]	validation_0-rmse:0.347903	validation_1-rmse:0.341081
[145]	validation_0-rmse:0.347659	validation_1-rmse:0.340931
[146]	validation_0-rmse:0.347332	validation_1-rmse:0.340695
[147]	validation_0-rmse:0.347128	validation_1-rmse:0.340519
[148]	validation_0-rmse:0.34663	validation_1-rmse:0.340224
[149]	validation_0-rmse:0.346533	validation_1-rmse:0.34011
[150]	validation_0-rmse:0.346423	validation_1-rmse:0.34013
[151]	validation_0-rmse:0.346305	validation_1-rmse:0.340081
[152]	validation_0-rmse:0.346148	validation_1-

[274]	validation_0-rmse:0.318064	validation_1-rmse:0.31598
[275]	validation_0-rmse:0.317525	validation_1-rmse:0.315523
[276]	validation_0-rmse:0.31746	validation_1-rmse:0.31548
[277]	validation_0-rmse:0.317186	validation_1-rmse:0.315318
[278]	validation_0-rmse:0.31708	validation_1-rmse:0.315219
[279]	validation_0-rmse:0.317027	validation_1-rmse:0.315216
[280]	validation_0-rmse:0.316784	validation_1-rmse:0.314964
[281]	validation_0-rmse:0.31669	validation_1-rmse:0.314873
[282]	validation_0-rmse:0.316437	validation_1-rmse:0.31466
[283]	validation_0-rmse:0.316252	validation_1-rmse:0.314513
[284]	validation_0-rmse:0.316183	validation_1-rmse:0.31448
[285]	validation_0-rmse:0.316111	validation_1-rmse:0.314537
[286]	validation_0-rmse:0.316041	validation_1-rmse:0.314451
[287]	validation_0-rmse:0.315916	validation_1-rmse:0.314271
[288]	validation_0-rmse:0.315749	validation_1-rmse:0.314076
[289]	validation_0-rmse:0.315695	validation_1-rmse:0.314027
[290]	validation_0-rmse:0.315605	validation_1-r

[412]	validation_0-rmse:0.291109	validation_1-rmse:0.29276
[413]	validation_0-rmse:0.291065	validation_1-rmse:0.292718
[414]	validation_0-rmse:0.290868	validation_1-rmse:0.292533
[415]	validation_0-rmse:0.290849	validation_1-rmse:0.292519
[416]	validation_0-rmse:0.290569	validation_1-rmse:0.292237
[417]	validation_0-rmse:0.29034	validation_1-rmse:0.292009
[418]	validation_0-rmse:0.290109	validation_1-rmse:0.291757
[419]	validation_0-rmse:0.290038	validation_1-rmse:0.291731
[420]	validation_0-rmse:0.289794	validation_1-rmse:0.291525
[421]	validation_0-rmse:0.289612	validation_1-rmse:0.291363
[422]	validation_0-rmse:0.289429	validation_1-rmse:0.291244
[423]	validation_0-rmse:0.289174	validation_1-rmse:0.291014
[424]	validation_0-rmse:0.289136	validation_1-rmse:0.291013
[425]	validation_0-rmse:0.288991	validation_1-rmse:0.290913
[426]	validation_0-rmse:0.28892	validation_1-rmse:0.290863
[427]	validation_0-rmse:0.288732	validation_1-rmse:0.2907
[428]	validation_0-rmse:0.288456	validation_1

[550]	validation_0-rmse:0.27189	validation_1-rmse:0.275499
[551]	validation_0-rmse:0.271834	validation_1-rmse:0.275488
[552]	validation_0-rmse:0.271762	validation_1-rmse:0.275395
[553]	validation_0-rmse:0.271704	validation_1-rmse:0.275358
[554]	validation_0-rmse:0.271483	validation_1-rmse:0.27511
[555]	validation_0-rmse:0.271374	validation_1-rmse:0.275004
[556]	validation_0-rmse:0.271144	validation_1-rmse:0.274779
[557]	validation_0-rmse:0.271094	validation_1-rmse:0.274776
[558]	validation_0-rmse:0.270996	validation_1-rmse:0.274698
[559]	validation_0-rmse:0.270758	validation_1-rmse:0.274468
[560]	validation_0-rmse:0.270505	validation_1-rmse:0.274197
[561]	validation_0-rmse:0.270287	validation_1-rmse:0.273993
[562]	validation_0-rmse:0.270168	validation_1-rmse:0.273901
[563]	validation_0-rmse:0.269883	validation_1-rmse:0.273601
[564]	validation_0-rmse:0.269833	validation_1-rmse:0.273505
[565]	validation_0-rmse:0.269639	validation_1-rmse:0.273375
[566]	validation_0-rmse:0.269372	validatio

...

[4204]	validation_0-rmse:0.149696	validation_1-rmse:0.17531
[4205]	validation_0-rmse:0.149692	validation_1-rmse:0.175315
[4206]	validation_0-rmse:0.149687	validation_1-rmse:0.175307
[4207]	validation_0-rmse:0.149677	validation_1-rmse:0.175295
[4208]	validation_0-rmse:0.149672	validation_1-rmse:0.175293
[4209]	validation_0-rmse:0.14966	validation_1-rmse:0.175274
[4210]	validation_0-rmse:0.149656	validation_1-rmse:0.175287
[4211]	validation_0-rmse:0.149653	validation_1-rmse:0.175286
[4212]	validation_0-rmse:0.149651	validation_1-rmse:0.175282
[4213]	validation_0-rmse:0.149646	validation_1-rmse:0.175288
[4214]	validation_0-rmse:0.149642	validation_1-rmse:0.175289
[4215]	validation_0-rmse:0.149638	validation_1-rmse:0.175293
[4216]	validation_0-rmse:0.149632	validation_1-rmse:0.175285
[4217]	validation_0-rmse:0.149627	validation_1-rmse:0.175281
[4218]	validation_0-rmse:0.149626	validation_1-rmse:0.175278
[4219]	validation_0-rmse:0.149622	validation_1-rmse:0.175272
[4220]	validation_0-rmse:0

[4339]	validation_0-rmse:0.148819	validation_1-rmse:0.174728
[4340]	validation_0-rmse:0.148807	validation_1-rmse:0.174735
[4341]	validation_0-rmse:0.148804	validation_1-rmse:0.174743
[4342]	validation_0-rmse:0.148798	validation_1-rmse:0.174755
[4343]	validation_0-rmse:0.148794	validation_1-rmse:0.174748
[4344]	validation_0-rmse:0.148787	validation_1-rmse:0.174748
[4345]	validation_0-rmse:0.148779	validation_1-rmse:0.174732
[4346]	validation_0-rmse:0.148773	validation_1-rmse:0.174729
[4347]	validation_0-rmse:0.148726	validation_1-rmse:0.174695
[4348]	validation_0-rmse:0.148719	validation_1-rmse:0.1747
[4349]	validation_0-rmse:0.148714	validation_1-rmse:0.174699
[4350]	validation_0-rmse:0.14871	validation_1-rmse:0.174693
[4351]	validation_0-rmse:0.148705	validation_1-rmse:0.174695
[4352]	validation_0-rmse:0.148699	validation_1-rmse:0.174694
[4353]	validation_0-rmse:0.14869	validation_1-rmse:0.174684
[4354]	validation_0-rmse:0.148682	validation_1-rmse:0.174678
[4355]	validation_0-rmse:0.1

[4474]	validation_0-rmse:0.147913	validation_1-rmse:0.174071
[4475]	validation_0-rmse:0.147905	validation_1-rmse:0.174067
[4476]	validation_0-rmse:0.147899	validation_1-rmse:0.174052
[4477]	validation_0-rmse:0.147895	validation_1-rmse:0.174063
[4478]	validation_0-rmse:0.147887	validation_1-rmse:0.174058
[4479]	validation_0-rmse:0.147885	validation_1-rmse:0.174052
[4480]	validation_0-rmse:0.147881	validation_1-rmse:0.174046
[4481]	validation_0-rmse:0.147877	validation_1-rmse:0.174041
[4482]	validation_0-rmse:0.147854	validation_1-rmse:0.174029
[4483]	validation_0-rmse:0.147849	validation_1-rmse:0.174041
[4484]	validation_0-rmse:0.147844	validation_1-rmse:0.174039
[4485]	validation_0-rmse:0.147842	validation_1-rmse:0.174036
[4486]	validation_0-rmse:0.147834	validation_1-rmse:0.174023
[4487]	validation_0-rmse:0.14783	validation_1-rmse:0.174021
[4488]	validation_0-rmse:0.147826	validation_1-rmse:0.17403
[4489]	validation_0-rmse:0.147821	validation_1-rmse:0.174022
[4490]	validation_0-rmse:0

[4609]	validation_0-rmse:0.147097	validation_1-rmse:0.173698
[4610]	validation_0-rmse:0.147087	validation_1-rmse:0.173695
[4611]	validation_0-rmse:0.147085	validation_1-rmse:0.173689
[4612]	validation_0-rmse:0.147079	validation_1-rmse:0.173676
[4613]	validation_0-rmse:0.147078	validation_1-rmse:0.173672
[4614]	validation_0-rmse:0.147075	validation_1-rmse:0.17367
[4615]	validation_0-rmse:0.14707	validation_1-rmse:0.173655
[4616]	validation_0-rmse:0.147066	validation_1-rmse:0.173661
[4617]	validation_0-rmse:0.147059	validation_1-rmse:0.173676
[4618]	validation_0-rmse:0.147051	validation_1-rmse:0.173669
[4619]	validation_0-rmse:0.147049	validation_1-rmse:0.173662
[4620]	validation_0-rmse:0.147042	validation_1-rmse:0.173653
[4621]	validation_0-rmse:0.147038	validation_1-rmse:0.173649
[4622]	validation_0-rmse:0.147034	validation_1-rmse:0.173655
[4623]	validation_0-rmse:0.147032	validation_1-rmse:0.173646
[4624]	validation_0-rmse:0.147026	validation_1-rmse:0.173637
[4625]	validation_0-rmse:0

[4744]	validation_0-rmse:0.146368	validation_1-rmse:0.17312
[4745]	validation_0-rmse:0.146361	validation_1-rmse:0.173118
[4746]	validation_0-rmse:0.146358	validation_1-rmse:0.173119
[4747]	validation_0-rmse:0.146353	validation_1-rmse:0.173117
[4748]	validation_0-rmse:0.146349	validation_1-rmse:0.173102
[4749]	validation_0-rmse:0.146346	validation_1-rmse:0.173101
[4750]	validation_0-rmse:0.146337	validation_1-rmse:0.173095
[4751]	validation_0-rmse:0.14633	validation_1-rmse:0.173088
[4752]	validation_0-rmse:0.146325	validation_1-rmse:0.173088
[4753]	validation_0-rmse:0.146322	validation_1-rmse:0.173086
[4754]	validation_0-rmse:0.146319	validation_1-rmse:0.173081
[4755]	validation_0-rmse:0.146318	validation_1-rmse:0.173071
[4756]	validation_0-rmse:0.14631	validation_1-rmse:0.173063
[4757]	validation_0-rmse:0.146309	validation_1-rmse:0.173056
[4758]	validation_0-rmse:0.146304	validation_1-rmse:0.173049
[4759]	validation_0-rmse:0.146297	validation_1-rmse:0.173039
[4760]	validation_0-rmse:0.

In [20]:
# Saving the predicted values
y_pred_1 = gbm_fit.predict(x_test_1, ntree_limit=gbm_fit.best_ntree_limit)

In [22]:
np.sqrt(mean_squared_error(y_test_1, y_pred_1))

0.17277150026697735

In [23]:
# Tracking evaluation error for training and testing set 2
eval_set_2 = [(x_train_2, y_train_2),
              (x_test_2, y_test_2)]

In [24]:
gbm_fit = gbm.fit(x_train_2,
                  y_train_2,
                  eval_set=eval_set_2,
                  eval_metric='rmse',
                  early_stopping_rounds=50,
                  verbose=True)

[17:14:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:7.84235	validation_1-rmse:7.88612
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:7.45127	validation_1-rmse:7.49487
[2]	validation_0-rmse:7.0798	validation_1-rmse:7.12356
[3]	validation_0-rmse:6.72693	validation_1-rmse:6.77056
[4]	validation_0-rmse:6.39178	validation_1-rmse:6.43528
[5]	validation_0-rmse:6.07346	validation_1-rmse:6.11686
[6]	validation_0-rmse:5.77111	validation_1-rmse:5.81462
[7]	validation_0-rmse:5.48393	validation_1-rmse:5.52733
[8]	validation_0-rmse:5.2112	validation_1-rmse:5.2546
[9]	validation_0-rmse:4.95222	validation_1-rmse:4.99556
[10]	validation_0-rmse:4.7062	validation_1-rmse:4.74996
[11]	validation_0-rmse:4.47255	validation_1-rmse:4.51641
[12]	validation_0-rmse:4.25064	validation_1-

[136]	validation_0-rmse:0.349112	validation_1-rmse:0.347585
[137]	validation_0-rmse:0.348986	validation_1-rmse:0.347385
[138]	validation_0-rmse:0.348356	validation_1-rmse:0.346746
[139]	validation_0-rmse:0.348052	validation_1-rmse:0.346338
[140]	validation_0-rmse:0.34787	validation_1-rmse:0.346144
[141]	validation_0-rmse:0.347627	validation_1-rmse:0.345911
[142]	validation_0-rmse:0.347459	validation_1-rmse:0.34572
[143]	validation_0-rmse:0.34733	validation_1-rmse:0.345517
[144]	validation_0-rmse:0.34707	validation_1-rmse:0.345333
[145]	validation_0-rmse:0.34664	validation_1-rmse:0.344854
[146]	validation_0-rmse:0.346382	validation_1-rmse:0.34468
[147]	validation_0-rmse:0.346227	validation_1-rmse:0.34462
[148]	validation_0-rmse:0.345997	validation_1-rmse:0.344405
[149]	validation_0-rmse:0.3459	validation_1-rmse:0.344244
[150]	validation_0-rmse:0.345477	validation_1-rmse:0.343824
[151]	validation_0-rmse:0.345345	validation_1-rmse:0.343671
[152]	validation_0-rmse:0.345058	validation_1-rms

[274]	validation_0-rmse:0.317549	validation_1-rmse:0.318766
[275]	validation_0-rmse:0.31721	validation_1-rmse:0.318486
[276]	validation_0-rmse:0.317152	validation_1-rmse:0.318405
[277]	validation_0-rmse:0.31693	validation_1-rmse:0.318206
[278]	validation_0-rmse:0.316862	validation_1-rmse:0.318025
[279]	validation_0-rmse:0.3167	validation_1-rmse:0.317859
[280]	validation_0-rmse:0.316655	validation_1-rmse:0.31784
[281]	validation_0-rmse:0.316506	validation_1-rmse:0.317702
[282]	validation_0-rmse:0.31614	validation_1-rmse:0.317377
[283]	validation_0-rmse:0.315939	validation_1-rmse:0.317202
[284]	validation_0-rmse:0.31568	validation_1-rmse:0.316991
[285]	validation_0-rmse:0.31554	validation_1-rmse:0.316955
[286]	validation_0-rmse:0.315225	validation_1-rmse:0.316703
[287]	validation_0-rmse:0.314996	validation_1-rmse:0.316494
[288]	validation_0-rmse:0.314924	validation_1-rmse:0.316466
[289]	validation_0-rmse:0.314835	validation_1-rmse:0.316521
[290]	validation_0-rmse:0.314765	validation_1-rm

[412]	validation_0-rmse:0.292927	validation_1-rmse:0.296266
[413]	validation_0-rmse:0.292751	validation_1-rmse:0.296122
[414]	validation_0-rmse:0.29269	validation_1-rmse:0.29607
[415]	validation_0-rmse:0.292502	validation_1-rmse:0.295971
[416]	validation_0-rmse:0.292204	validation_1-rmse:0.29568
[417]	validation_0-rmse:0.292127	validation_1-rmse:0.295594
[418]	validation_0-rmse:0.292092	validation_1-rmse:0.295551
[419]	validation_0-rmse:0.291881	validation_1-rmse:0.295317
[420]	validation_0-rmse:0.291841	validation_1-rmse:0.295226
[421]	validation_0-rmse:0.291636	validation_1-rmse:0.295058
[422]	validation_0-rmse:0.2915	validation_1-rmse:0.294915
[423]	validation_0-rmse:0.291462	validation_1-rmse:0.294833
[424]	validation_0-rmse:0.291363	validation_1-rmse:0.294761
[425]	validation_0-rmse:0.291127	validation_1-rmse:0.294555
[426]	validation_0-rmse:0.290913	validation_1-rmse:0.294393
[427]	validation_0-rmse:0.290845	validation_1-rmse:0.294283
[428]	validation_0-rmse:0.290651	validation_1

[550]	validation_0-rmse:0.273195	validation_1-rmse:0.278434
[551]	validation_0-rmse:0.273156	validation_1-rmse:0.278418
[552]	validation_0-rmse:0.273116	validation_1-rmse:0.278385
[553]	validation_0-rmse:0.27307	validation_1-rmse:0.278338
[554]	validation_0-rmse:0.272913	validation_1-rmse:0.27818
[555]	validation_0-rmse:0.272621	validation_1-rmse:0.277904
[556]	validation_0-rmse:0.272575	validation_1-rmse:0.277808
[557]	validation_0-rmse:0.272424	validation_1-rmse:0.27768
[558]	validation_0-rmse:0.272406	validation_1-rmse:0.277686
[559]	validation_0-rmse:0.272356	validation_1-rmse:0.277675
[560]	validation_0-rmse:0.272262	validation_1-rmse:0.277616
[561]	validation_0-rmse:0.272224	validation_1-rmse:0.27753
[562]	validation_0-rmse:0.272052	validation_1-rmse:0.277413
[563]	validation_0-rmse:0.272	validation_1-rmse:0.277334
[564]	validation_0-rmse:0.271842	validation_1-rmse:0.277163
[565]	validation_0-rmse:0.271803	validation_1-rmse:0.277121
[566]	validation_0-rmse:0.271682	validation_1-r

...

[4338]	validation_0-rmse:0.150732	validation_1-rmse:0.17219
[4339]	validation_0-rmse:0.150725	validation_1-rmse:0.172187
[4340]	validation_0-rmse:0.15072	validation_1-rmse:0.172192
[4341]	validation_0-rmse:0.15072	validation_1-rmse:0.172191
[4342]	validation_0-rmse:0.150719	validation_1-rmse:0.172187
[4343]	validation_0-rmse:0.150716	validation_1-rmse:0.172186
[4344]	validation_0-rmse:0.150712	validation_1-rmse:0.172179
[4345]	validation_0-rmse:0.150706	validation_1-rmse:0.172174
[4346]	validation_0-rmse:0.1507	validation_1-rmse:0.17217
[4347]	validation_0-rmse:0.150697	validation_1-rmse:0.172167
[4348]	validation_0-rmse:0.150694	validation_1-rmse:0.172164
[4349]	validation_0-rmse:0.150692	validation_1-rmse:0.172165
[4350]	validation_0-rmse:0.150687	validation_1-rmse:0.172163
[4351]	validation_0-rmse:0.150683	validation_1-rmse:0.17216
[4352]	validation_0-rmse:0.150674	validation_1-rmse:0.172154
[4353]	validation_0-rmse:0.150661	validation_1-rmse:0.172147
[4354]	validation_0-rmse:0.1506

[4473]	validation_0-rmse:0.149884	validation_1-rmse:0.171637
[4474]	validation_0-rmse:0.14988	validation_1-rmse:0.171636
[4475]	validation_0-rmse:0.149872	validation_1-rmse:0.171631
[4476]	validation_0-rmse:0.149867	validation_1-rmse:0.171633
[4477]	validation_0-rmse:0.149865	validation_1-rmse:0.171625
[4478]	validation_0-rmse:0.149858	validation_1-rmse:0.171621
[4479]	validation_0-rmse:0.149849	validation_1-rmse:0.171612
[4480]	validation_0-rmse:0.149843	validation_1-rmse:0.1716
[4481]	validation_0-rmse:0.149836	validation_1-rmse:0.171585
[4482]	validation_0-rmse:0.149831	validation_1-rmse:0.171577
[4483]	validation_0-rmse:0.149826	validation_1-rmse:0.17157
[4484]	validation_0-rmse:0.149819	validation_1-rmse:0.171562
[4485]	validation_0-rmse:0.149813	validation_1-rmse:0.171576
[4486]	validation_0-rmse:0.149803	validation_1-rmse:0.171555
[4487]	validation_0-rmse:0.149794	validation_1-rmse:0.171545
[4488]	validation_0-rmse:0.149787	validation_1-rmse:0.171537
[4489]	validation_0-rmse:0.1

[4608]	validation_0-rmse:0.149089	validation_1-rmse:0.171249
[4609]	validation_0-rmse:0.149078	validation_1-rmse:0.171237
[4610]	validation_0-rmse:0.149073	validation_1-rmse:0.17123
[4611]	validation_0-rmse:0.149063	validation_1-rmse:0.171218
[4612]	validation_0-rmse:0.149058	validation_1-rmse:0.171201
[4613]	validation_0-rmse:0.149056	validation_1-rmse:0.171195
[4614]	validation_0-rmse:0.149046	validation_1-rmse:0.171186
[4615]	validation_0-rmse:0.149039	validation_1-rmse:0.171188
[4616]	validation_0-rmse:0.149034	validation_1-rmse:0.171188
[4617]	validation_0-rmse:0.149027	validation_1-rmse:0.171182
[4618]	validation_0-rmse:0.149023	validation_1-rmse:0.171173
[4619]	validation_0-rmse:0.149019	validation_1-rmse:0.171162
[4620]	validation_0-rmse:0.149014	validation_1-rmse:0.171165
[4621]	validation_0-rmse:0.148974	validation_1-rmse:0.17113
[4622]	validation_0-rmse:0.148968	validation_1-rmse:0.171129
[4623]	validation_0-rmse:0.148967	validation_1-rmse:0.171131
[4624]	validation_0-rmse:0

[4743]	validation_0-rmse:0.148253	validation_1-rmse:0.170623
[4744]	validation_0-rmse:0.148243	validation_1-rmse:0.170614
[4745]	validation_0-rmse:0.148236	validation_1-rmse:0.170599
[4746]	validation_0-rmse:0.14823	validation_1-rmse:0.170594
[4747]	validation_0-rmse:0.148226	validation_1-rmse:0.170591
[4748]	validation_0-rmse:0.148225	validation_1-rmse:0.170592
[4749]	validation_0-rmse:0.148221	validation_1-rmse:0.170592
[4750]	validation_0-rmse:0.148216	validation_1-rmse:0.170584
[4751]	validation_0-rmse:0.14821	validation_1-rmse:0.170576
[4752]	validation_0-rmse:0.148206	validation_1-rmse:0.170567
[4753]	validation_0-rmse:0.148194	validation_1-rmse:0.170563
[4754]	validation_0-rmse:0.148188	validation_1-rmse:0.170556
[4755]	validation_0-rmse:0.148173	validation_1-rmse:0.170542
[4756]	validation_0-rmse:0.148166	validation_1-rmse:0.170538
[4757]	validation_0-rmse:0.148163	validation_1-rmse:0.170534
[4758]	validation_0-rmse:0.148154	validation_1-rmse:0.170525
[4759]	validation_0-rmse:0

[4878]	validation_0-rmse:0.147499	validation_1-rmse:0.170385
[4879]	validation_0-rmse:0.147492	validation_1-rmse:0.170383
[4880]	validation_0-rmse:0.147491	validation_1-rmse:0.170381
[4881]	validation_0-rmse:0.147481	validation_1-rmse:0.170362
[4882]	validation_0-rmse:0.147456	validation_1-rmse:0.17034
[4883]	validation_0-rmse:0.147452	validation_1-rmse:0.170336
[4884]	validation_0-rmse:0.14745	validation_1-rmse:0.170331
[4885]	validation_0-rmse:0.147441	validation_1-rmse:0.170327
[4886]	validation_0-rmse:0.14744	validation_1-rmse:0.170326
[4887]	validation_0-rmse:0.147438	validation_1-rmse:0.170324
[4888]	validation_0-rmse:0.147437	validation_1-rmse:0.170325
[4889]	validation_0-rmse:0.147428	validation_1-rmse:0.170318
[4890]	validation_0-rmse:0.147424	validation_1-rmse:0.170314
[4891]	validation_0-rmse:0.147423	validation_1-rmse:0.170313
[4892]	validation_0-rmse:0.14742	validation_1-rmse:0.17031
[4893]	validation_0-rmse:0.147418	validation_1-rmse:0.170292
[4894]	validation_0-rmse:0.14

In [25]:
# Saving the predicted values for the second testing dataset
y_pred_2 = gbm_fit.predict(x_test_2, ntree_limit=gbm_fit.best_ntree_limit)

In [26]:
# Calculating RMSE
np.sqrt(mean_squared_error(y_test_2, y_pred_2))

0.1701974449433926

Comparing the RMSE to the average RMSE (~0.11) of our SARIMA model, we can see that the SARIMA model performs better than a basic XGBoost model.

Now we will retrain our model with more features that we have engineered.

### XGBoost with feature engineering

In [28]:
with open('data_with_feature_engineering.pickle', 'rb') as f:
    df_extended = pickle.load(f)

In [29]:
df_extended.head()

,Date,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,IsHoliday,StoreType,Assortment,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,LogCompetitionDistance,StorePermutation,LogSales,CompetitionTenureMonths,LogCompetitionMonths,Promo2Months,LogPromo2Months,Lagged1LogSales,Lagged2LogSales,Lagged3LogSales,Lagged4LogSales,Lagged5LogSales,Lagged6LogSales,Lagged7LogSales,Lagged8LogSales,Lagged9LogSales,Lagged10LogSales,Lagged11LogSales,Lagged12LogSales,Past6daysAvgLogSales,Past12DaysAvgLogSales,Past6DaysLogSalesSTDEV,Past12DaysLogSalesSTDEV,Past6DaysMaxLogSales,Past12DaysMaxLogSales,Past6DaysMinLogSales,Past12DaysMinLogSales
0,2013-01-01,1,2,5797.0,0,0,1,1,2013,1,1,1,1,3,1,9,2008,0,0,0,0,7.146772,7,8.665268,52,3.970292,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,3,5530.0,1,0,0,1,2013,1,2,1,1,3,1,9,2008,0,0,0,0,7.146772,7,8.618124,52,3.970292,0.0,0.0,8.665268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,4,4327.0,1,0,0,1,2013,1,3,1,1,3,1,9,2008,0,0,0,0,7.146772,7,8.372861,52,3.970292,0.0,0.0,8.618124,8.665268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,5,4486.0,1,0,0,1,2013,1,4,1,1,3,1,9,2008,0,0,0,0,7.146772,7,8.408940,52,3.970292,0.0,0.0,8.372861,8.618124,8.665268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,6,4997.0,1,0,0,1,2013,1,5,1,1,3,1,9,2008,0,0,0,0,7.146772,7,8.516793,52,3.970292,0.0,0.0,8.408940,8.372861,8.618124,8.665268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_extended.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear', 'IsHoliday',
       'StoreType', 'Assortment', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'LogCompetitionDistance',
       'StorePermutation', 'LogSales', 'CompetitionTenureMonths',
       'LogCompetitionMonths', 'Promo2Months', 'LogPromo2Months',
       'Lagged1LogSales', 'Lagged2LogSales', 'Lagged3LogSales',
       'Lagged4LogSales', 'Lagged5LogSales', 'Lagged6LogSales',
       'Lagged7LogSales', 'Lagged8LogSales', 'Lagged9LogSales',
       'Lagged10LogSales', 'Lagged11LogSales', 'Lagged12LogSales',
       'Past6daysAvgLogSales', 'Past12DaysAvgLogSales',
       'Past6DaysLogSalesSTDEV', 'Past12DaysLogSalesSTDEV',
       'Past6DaysMaxLogSales', 'Past12DaysMaxLogSales', 'Past6DaysMinLogSales',
       'Past12DaysMinLogSales'],
      dtype='object')

In [31]:
df_extended.drop(['Sales', 'Promo2Months', 'CompetitionTenureMonths',
                  'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                  'Promo2SinceWeek', 'Promo2SinceYear'], axis=1, inplace=True)

In [32]:
df_extended.dropna(inplace=True)

In [33]:
df_extended.shape

(855066, 41)

In [34]:
df_extended = df_extended.set_index('Date').sort_index(ascending=True)

In [35]:
df_extended = df_extended.loc[df_extended.StoreType == 1]

In [36]:
trainset_x1, testset_x1, trainset_x2, testset_x2 = train_test_split(df_extended)

Initial dataset has been split into two sets of training and testing datasets each.
Trainset 1 is from 2013-01-02 to 2014-12-31 : N_obs = 355708
Testset 1 is from 2015-01-01 to 2015-03-31 : N_obs = 46275


Trainset 2 is from 2013-01-02 to 2015-03-31 : N_obs = 401983
Testset 2 is from 2015-04-01 to 2015-07-31 : N_obs = 63105


In [37]:
y_train_x1 = np.array(trainset_x1.LogSales)
x_train_x1 = np.array(trainset_x1.drop('LogSales', axis=1))

y_train_x2 = np.array(trainset_x2.LogSales)
x_train_x2 = np.array(trainset_x2.drop('LogSales', axis=1))

y_test_x1 = np.array(testset_x1.LogSales)
x_test_x1 = np.array(testset_x1.drop('LogSales', axis=1))

y_test_x2 = np.array(testset_x2.LogSales)
x_test_x2 = np.array(testset_x2.drop('LogSales', axis=1))

In [38]:
# Tracking evaluation error for training and testing set 1
eval_set_x1 = [(x_train_x1, y_train_x1),
              (x_test_x1, y_test_x1)]

In [39]:
gbm_fit = gbm.fit(x_train_x1, 
                  y_train_x1,
                  eval_set=eval_set_x1,
                  eval_metric='rmse',
                  early_stopping_rounds=50,
                  verbose=True)

[17:31:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:7.84344	validation_1-rmse:7.83064
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:7.45167	validation_1-rmse:7.43945
[2]	validation_0-rmse:7.07952	validation_1-rmse:7.06773
[3]	validation_0-rmse:6.72597	validation_1-rmse:6.71459
[4]	validation_0-rmse:6.39011	validation_1-rmse:6.37929
[5]	validation_0-rmse:6.07106	validation_1-rmse:6.06073
[6]	validation_0-rmse:5.76795	validation_1-rmse:5.75822
[7]	validation_0-rmse:5.48002	validation_1-rmse:5.47076
[8]	validation_0-rmse:5.20646	validation_1-rmse:5.19726
[9]	validation_0-rmse:4.94664	validation_1-rmse:4.93787
[10]	validation_0-rmse:4.69982	validation_1-rmse:4.69146
[11]	validation_0-rmse:4.46536	validation_1-rmse:4.45715
[12]	validation_0-rmse:4.24263	validatio

[136]	validation_0-rmse:0.144028	validation_1-rmse:0.148622
[137]	validation_0-rmse:0.143902	validation_1-rmse:0.148475
[138]	validation_0-rmse:0.143784	validation_1-rmse:0.1484
[139]	validation_0-rmse:0.143678	validation_1-rmse:0.148347
[140]	validation_0-rmse:0.143436	validation_1-rmse:0.148349
[141]	validation_0-rmse:0.143328	validation_1-rmse:0.148232
[142]	validation_0-rmse:0.143224	validation_1-rmse:0.148181
[143]	validation_0-rmse:0.143119	validation_1-rmse:0.148125
[144]	validation_0-rmse:0.142999	validation_1-rmse:0.148113
[145]	validation_0-rmse:0.142916	validation_1-rmse:0.148007
[146]	validation_0-rmse:0.142853	validation_1-rmse:0.14796
[147]	validation_0-rmse:0.142584	validation_1-rmse:0.147941
[148]	validation_0-rmse:0.142469	validation_1-rmse:0.147828
[149]	validation_0-rmse:0.142389	validation_1-rmse:0.147743
[150]	validation_0-rmse:0.142296	validation_1-rmse:0.147678
[151]	validation_0-rmse:0.142216	validation_1-rmse:0.147651
[152]	validation_0-rmse:0.142111	validation

[274]	validation_0-rmse:0.132782	validation_1-rmse:0.142451
[275]	validation_0-rmse:0.132679	validation_1-rmse:0.142412
[276]	validation_0-rmse:0.132635	validation_1-rmse:0.142363
[277]	validation_0-rmse:0.132553	validation_1-rmse:0.142267
[278]	validation_0-rmse:0.132511	validation_1-rmse:0.14228
[279]	validation_0-rmse:0.132455	validation_1-rmse:0.142297
[280]	validation_0-rmse:0.132429	validation_1-rmse:0.142254
[281]	validation_0-rmse:0.132374	validation_1-rmse:0.1422
[282]	validation_0-rmse:0.132344	validation_1-rmse:0.142178
[283]	validation_0-rmse:0.132306	validation_1-rmse:0.142155
[284]	validation_0-rmse:0.132269	validation_1-rmse:0.142082
[285]	validation_0-rmse:0.132175	validation_1-rmse:0.142057
[286]	validation_0-rmse:0.13213	validation_1-rmse:0.142026
[287]	validation_0-rmse:0.132067	validation_1-rmse:0.142043
[288]	validation_0-rmse:0.132055	validation_1-rmse:0.14204
[289]	validation_0-rmse:0.132019	validation_1-rmse:0.141995
[290]	validation_0-rmse:0.131992	validation_1

[412]	validation_0-rmse:0.127028	validation_1-rmse:0.138334
[413]	validation_0-rmse:0.12701	validation_1-rmse:0.138318
[414]	validation_0-rmse:0.126987	validation_1-rmse:0.138313
[415]	validation_0-rmse:0.126976	validation_1-rmse:0.138312
[416]	validation_0-rmse:0.126963	validation_1-rmse:0.138283
[417]	validation_0-rmse:0.126943	validation_1-rmse:0.138287
[418]	validation_0-rmse:0.12692	validation_1-rmse:0.138264
[419]	validation_0-rmse:0.126902	validation_1-rmse:0.138252
[420]	validation_0-rmse:0.126885	validation_1-rmse:0.138244
[421]	validation_0-rmse:0.126873	validation_1-rmse:0.138242
[422]	validation_0-rmse:0.126818	validation_1-rmse:0.138177
[423]	validation_0-rmse:0.126782	validation_1-rmse:0.138123
[424]	validation_0-rmse:0.126763	validation_1-rmse:0.138109
[425]	validation_0-rmse:0.126749	validation_1-rmse:0.13808
[426]	validation_0-rmse:0.126723	validation_1-rmse:0.13808
[427]	validation_0-rmse:0.126687	validation_1-rmse:0.138078
[428]	validation_0-rmse:0.126667	validation_

[550]	validation_0-rmse:0.123553	validation_1-rmse:0.136039
[551]	validation_0-rmse:0.123541	validation_1-rmse:0.136034
[552]	validation_0-rmse:0.123511	validation_1-rmse:0.136007
[553]	validation_0-rmse:0.123491	validation_1-rmse:0.136001
[554]	validation_0-rmse:0.123476	validation_1-rmse:0.135988
[555]	validation_0-rmse:0.123466	validation_1-rmse:0.135981
[556]	validation_0-rmse:0.123447	validation_1-rmse:0.135969
[557]	validation_0-rmse:0.123413	validation_1-rmse:0.135947
[558]	validation_0-rmse:0.123399	validation_1-rmse:0.135922
[559]	validation_0-rmse:0.123373	validation_1-rmse:0.135905
[560]	validation_0-rmse:0.123362	validation_1-rmse:0.135886
[561]	validation_0-rmse:0.123335	validation_1-rmse:0.135879
[562]	validation_0-rmse:0.123318	validation_1-rmse:0.135862
[563]	validation_0-rmse:0.123313	validation_1-rmse:0.135842
[564]	validation_0-rmse:0.123284	validation_1-rmse:0.135843
[565]	validation_0-rmse:0.12325	validation_1-rmse:0.135797
[566]	validation_0-rmse:0.12324	validatio

...

[963]	validation_0-rmse:0.117355	validation_1-rmse:0.131891
[964]	validation_0-rmse:0.117334	validation_1-rmse:0.131917
[965]	validation_0-rmse:0.11732	validation_1-rmse:0.131907
[966]	validation_0-rmse:0.117315	validation_1-rmse:0.131908
[967]	validation_0-rmse:0.117312	validation_1-rmse:0.131907
[968]	validation_0-rmse:0.117306	validation_1-rmse:0.131908
[969]	validation_0-rmse:0.117299	validation_1-rmse:0.131891
[970]	validation_0-rmse:0.117291	validation_1-rmse:0.131888
[971]	validation_0-rmse:0.117277	validation_1-rmse:0.13188
[972]	validation_0-rmse:0.117267	validation_1-rmse:0.131879
[973]	validation_0-rmse:0.117259	validation_1-rmse:0.13188
[974]	validation_0-rmse:0.11725	validation_1-rmse:0.131875
[975]	validation_0-rmse:0.117245	validation_1-rmse:0.13187
[976]	validation_0-rmse:0.117228	validation_1-rmse:0.13188
[977]	validation_0-rmse:0.117217	validation_1-rmse:0.131871
[978]	validation_0-rmse:0.117214	validation_1-rmse:0.131871
[979]	validation_0-rmse:0.117197	validation_1-

[1099]	validation_0-rmse:0.116028	validation_1-rmse:0.131172
[1100]	validation_0-rmse:0.116026	validation_1-rmse:0.131168
[1101]	validation_0-rmse:0.116022	validation_1-rmse:0.131169
[1102]	validation_0-rmse:0.116011	validation_1-rmse:0.131159
[1103]	validation_0-rmse:0.116007	validation_1-rmse:0.131153
[1104]	validation_0-rmse:0.115997	validation_1-rmse:0.131142
[1105]	validation_0-rmse:0.115995	validation_1-rmse:0.131142
[1106]	validation_0-rmse:0.115993	validation_1-rmse:0.131142
[1107]	validation_0-rmse:0.115983	validation_1-rmse:0.13114
[1108]	validation_0-rmse:0.115978	validation_1-rmse:0.131129
[1109]	validation_0-rmse:0.11597	validation_1-rmse:0.131131
[1110]	validation_0-rmse:0.115965	validation_1-rmse:0.131126
[1111]	validation_0-rmse:0.115956	validation_1-rmse:0.131116
[1112]	validation_0-rmse:0.115948	validation_1-rmse:0.131114
[1113]	validation_0-rmse:0.115934	validation_1-rmse:0.131105
[1114]	validation_0-rmse:0.11593	validation_1-rmse:0.131107
[1115]	validation_0-rmse:0.

[1234]	validation_0-rmse:0.11506	validation_1-rmse:0.130501
[1235]	validation_0-rmse:0.115054	validation_1-rmse:0.130497
[1236]	validation_0-rmse:0.11505	validation_1-rmse:0.130495
[1237]	validation_0-rmse:0.115042	validation_1-rmse:0.130491
[1238]	validation_0-rmse:0.115039	validation_1-rmse:0.13049
[1239]	validation_0-rmse:0.115035	validation_1-rmse:0.130488
[1240]	validation_0-rmse:0.115025	validation_1-rmse:0.130473
[1241]	validation_0-rmse:0.115021	validation_1-rmse:0.130469
[1242]	validation_0-rmse:0.115007	validation_1-rmse:0.130492
[1243]	validation_0-rmse:0.114995	validation_1-rmse:0.130485
[1244]	validation_0-rmse:0.114993	validation_1-rmse:0.130485
[1245]	validation_0-rmse:0.114984	validation_1-rmse:0.130475
[1246]	validation_0-rmse:0.11498	validation_1-rmse:0.130472
[1247]	validation_0-rmse:0.114977	validation_1-rmse:0.130473
[1248]	validation_0-rmse:0.114972	validation_1-rmse:0.13047
[1249]	validation_0-rmse:0.114965	validation_1-rmse:0.130467
[1250]	validation_0-rmse:0.11

[1369]	validation_0-rmse:0.114108	validation_1-rmse:0.129841
[1370]	validation_0-rmse:0.1141	validation_1-rmse:0.129832
[1371]	validation_0-rmse:0.11407	validation_1-rmse:0.129823
[1372]	validation_0-rmse:0.114065	validation_1-rmse:0.12982
[1373]	validation_0-rmse:0.114059	validation_1-rmse:0.12982
[1374]	validation_0-rmse:0.114051	validation_1-rmse:0.129819
[1375]	validation_0-rmse:0.114049	validation_1-rmse:0.129817
[1376]	validation_0-rmse:0.114043	validation_1-rmse:0.129811
[1377]	validation_0-rmse:0.114019	validation_1-rmse:0.129815
[1378]	validation_0-rmse:0.114014	validation_1-rmse:0.129802
[1379]	validation_0-rmse:0.114011	validation_1-rmse:0.129802
[1380]	validation_0-rmse:0.114008	validation_1-rmse:0.129802
[1381]	validation_0-rmse:0.114004	validation_1-rmse:0.129798
[1382]	validation_0-rmse:0.114	validation_1-rmse:0.129799
[1383]	validation_0-rmse:0.113997	validation_1-rmse:0.129798
[1384]	validation_0-rmse:0.113975	validation_1-rmse:0.129772
[1385]	validation_0-rmse:0.11397

[1504]	validation_0-rmse:0.113295	validation_1-rmse:0.129487
[1505]	validation_0-rmse:0.113291	validation_1-rmse:0.129484
[1506]	validation_0-rmse:0.113279	validation_1-rmse:0.129477
[1507]	validation_0-rmse:0.113273	validation_1-rmse:0.129478
[1508]	validation_0-rmse:0.113259	validation_1-rmse:0.129457
[1509]	validation_0-rmse:0.113257	validation_1-rmse:0.129458
[1510]	validation_0-rmse:0.113257	validation_1-rmse:0.129458
[1511]	validation_0-rmse:0.113256	validation_1-rmse:0.129458
[1512]	validation_0-rmse:0.11325	validation_1-rmse:0.129461
[1513]	validation_0-rmse:0.113248	validation_1-rmse:0.12946
[1514]	validation_0-rmse:0.113247	validation_1-rmse:0.129459
[1515]	validation_0-rmse:0.113241	validation_1-rmse:0.129458
[1516]	validation_0-rmse:0.113239	validation_1-rmse:0.129449
[1517]	validation_0-rmse:0.113235	validation_1-rmse:0.129449
[1518]	validation_0-rmse:0.113232	validation_1-rmse:0.129449
[1519]	validation_0-rmse:0.113228	validation_1-rmse:0.129446
[1520]	validation_0-rmse:0

In [40]:
# Saving the predicted values for testing dataset 1
y_pred_x1 = gbm_fit.predict(x_test_x1, ntree_limit=gbm_fit.best_ntree_limit)

In [41]:
# Tracking evaluation error for training and testing set 2
eval_set_x2 = [(x_train_x2, y_train_x2),
               (x_test_x2, y_test_x2)]

In [45]:
# Calculating the RMSE for testing data 1
np.sqrt(mean_squared_error(y_test_x1, y_pred_x1))

0.12938830355014252

In [42]:
gbm_fit = gbm.fit(x_train_x2, 
                    y_train_x2,
                    eval_set=eval_set_x2,
                    eval_metric='rmse',
                    early_stopping_rounds=50,
                    verbose=True)

[17:37:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:7.84199	validation_1-rmse:7.8834
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:7.45028	validation_1-rmse:7.48991
[2]	validation_0-rmse:7.0782	validation_1-rmse:7.11599
[3]	validation_0-rmse:6.7247	validation_1-rmse:6.76087
[4]	validation_0-rmse:6.38887	validation_1-rmse:6.42347
[5]	validation_0-rmse:6.06987	validation_1-rmse:6.10285
[6]	validation_0-rmse:5.76682	validation_1-rmse:5.79824
[7]	validation_0-rmse:5.47894	validation_1-rmse:5.50889
[8]	validation_0-rmse:5.20541	validation_1-rmse:5.23401
[9]	validation_0-rmse:4.94562	validation_1-rmse:4.97299
[10]	validation_0-rmse:4.69886	validation_1-rmse:4.72487
[11]	validation_0-rmse:4.46445	validation_1-rmse:4.48914
[12]	validation_0-rmse:4.24177	validation_1

[136]	validation_0-rmse:0.144567	validation_1-rmse:0.133053
[137]	validation_0-rmse:0.144343	validation_1-rmse:0.133007
[138]	validation_0-rmse:0.144215	validation_1-rmse:0.132922
[139]	validation_0-rmse:0.144054	validation_1-rmse:0.132888
[140]	validation_0-rmse:0.143955	validation_1-rmse:0.132825
[141]	validation_0-rmse:0.143886	validation_1-rmse:0.132778
[142]	validation_0-rmse:0.143598	validation_1-rmse:0.132669
[143]	validation_0-rmse:0.143469	validation_1-rmse:0.132577
[144]	validation_0-rmse:0.143344	validation_1-rmse:0.132495
[145]	validation_0-rmse:0.143233	validation_1-rmse:0.132449
[146]	validation_0-rmse:0.143143	validation_1-rmse:0.132392
[147]	validation_0-rmse:0.143071	validation_1-rmse:0.132356
[148]	validation_0-rmse:0.142966	validation_1-rmse:0.132395
[149]	validation_0-rmse:0.142917	validation_1-rmse:0.132348
[150]	validation_0-rmse:0.142782	validation_1-rmse:0.132276
[151]	validation_0-rmse:0.142672	validation_1-rmse:0.132267
[152]	validation_0-rmse:0.142461	validat

[273]	validation_0-rmse:0.132998	validation_1-rmse:0.12951
[274]	validation_0-rmse:0.132957	validation_1-rmse:0.129484
[275]	validation_0-rmse:0.132907	validation_1-rmse:0.129467
[276]	validation_0-rmse:0.132796	validation_1-rmse:0.129474
[277]	validation_0-rmse:0.132768	validation_1-rmse:0.129452
[278]	validation_0-rmse:0.132738	validation_1-rmse:0.129417
[279]	validation_0-rmse:0.132722	validation_1-rmse:0.129418
[280]	validation_0-rmse:0.132648	validation_1-rmse:0.129404
[281]	validation_0-rmse:0.132623	validation_1-rmse:0.129395
[282]	validation_0-rmse:0.132582	validation_1-rmse:0.129383
[283]	validation_0-rmse:0.132537	validation_1-rmse:0.129373
[284]	validation_0-rmse:0.132445	validation_1-rmse:0.1293
[285]	validation_0-rmse:0.132417	validation_1-rmse:0.129296
[286]	validation_0-rmse:0.132362	validation_1-rmse:0.129292
[287]	validation_0-rmse:0.132325	validation_1-rmse:0.129264
[288]	validation_0-rmse:0.132251	validation_1-rmse:0.129247
[289]	validation_0-rmse:0.132187	validation

In [46]:
# Saving the predicted values for testing dataset 1
y_pred_x2 = gbm_fit.predict(x_test_x2, ntree_limit=gbm_fit.best_ntree_limit)

In [48]:
# Calculating the data for testing dataset 2
np.sqrt(mean_squared_error(y_test_x2, y_pred_x2))

0.12844087670123

We can see that there was a large improvement to our model after feature engineering with an average RMSE of around 0.13, less than the previous 0.18.

Although it is still higher than the SARIMA model that we obtained, the model selection and training time was much lesser.

We are also able to predict sales on a more granular store basis as well.

Therefore, the optimal model to use would depend on the business need. 

If the company wants an understanding of how their sales varies over time, the SARIMA model would perform better. 

On the other hand, if the company wants a quick predictive model on a more granular level, then boosting algorithms such as XGBoost will be more appropriate.